In [1]:
import os
import cv2
import torch
import numpy as np
import pandas as pd

import wandb
import datetime
from tqdm.auto import tqdm

from timm.models import resnet18, efficientnetv2_rw_s, efficientnetv2_s
from torchsummary import summary

In [2]:
model = efficientnetv2_rw_s(pretrained=True).cuda()
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 64, 64]             648
          Identity-2           [-1, 24, 64, 64]               0
              SiLU-3           [-1, 24, 64, 64]               0
    BatchNormAct2d-4           [-1, 24, 64, 64]              48
            Conv2d-5           [-1, 24, 64, 64]           5,184
          Identity-6           [-1, 24, 64, 64]               0
              SiLU-7           [-1, 24, 64, 64]               0
    BatchNormAct2d-8           [-1, 24, 64, 64]              48
          Identity-9           [-1, 24, 64, 64]               0
           Conv2d-10           [-1, 24, 64, 64]             576
         Identity-11           [-1, 24, 64, 64]               0
         Identity-12           [-1, 24, 64, 64]               0
   BatchNormAct2d-13           [-1, 24, 64, 64]              48
         Identity-14           [-1, 24,

In [4]:
model = efficientnetv2_s().cuda()
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 64, 64]             648
          Identity-2           [-1, 24, 64, 64]               0
              SiLU-3           [-1, 24, 64, 64]               0
    BatchNormAct2d-4           [-1, 24, 64, 64]              48
            Conv2d-5           [-1, 24, 64, 64]           5,184
          Identity-6           [-1, 24, 64, 64]               0
              SiLU-7           [-1, 24, 64, 64]               0
    BatchNormAct2d-8           [-1, 24, 64, 64]              48
          Identity-9           [-1, 24, 64, 64]               0
        ConvBnAct-10           [-1, 24, 64, 64]               0
           Conv2d-11           [-1, 24, 64, 64]           5,184
         Identity-12           [-1, 24, 64, 64]               0
             SiLU-13           [-1, 24, 64, 64]               0
   BatchNormAct2d-14           [-1, 24,

In [5]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Seq

In [2]:
model = resnet18(pretrained=True).cuda()

summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
          Identity-7           [-1, 64, 32, 32]               0
              ReLU-8           [-1, 64, 32, 32]               0
          Identity-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
       BasicBlock-13           [-1, 64, 32, 32]               0
           Conv2d-14           [-1, 64,

In [3]:
df = pd.read_csv("/opt/ml/level3_cv_finalproject-cv-01/model/data/data.csv")
df.head()

,id,class_id,label,img_path,json_path
0,0,1,baek_sook,./data/image/baek_sook/baek_sook_0001.jpg,./data/json/baek_sook/1_korea_baek_sook.json
1,1,2,baek_sook,./data/image/baek_sook/baek_sook_0002.jpg,./data/json/baek_sook/2_korea_baek_sook.json
2,2,3,baek_sook,./data/image/baek_sook/baek_sook_0003.jpg,./data/json/baek_sook/3_korea_baek_sook.json
3,3,4,baek_sook,./data/image/baek_sook/baek_sook_0004.jpg,./data/json/baek_sook/4_korea_baek_sook.json
4,4,5,baek_sook,./data/image/baek_sook/baek_sook_0005.jpg,./data/json/baek_sook/5_korea_baek_sook.json


In [4]:
base_path = "/opt/ml/level3_cv_finalproject-cv-01/model"
images_path = []
labels = []

for i, image_path, label in zip(df['class_id'], df["img_path"], df["label"]):
    if i > 200:
        images_path.append(os.path.join(base_path, image_path))
        labels.append(label)

classes = sorted(set(labels))
cls2idx = {c:i for i, c in enumerate(classes)}
print(f"labels : {len(labels)}, classes : {len(classes)}")

labels : 76673, classes : 93


In [5]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_path, is_train=False, tf=None) -> None:
        df = pd.read_csv(csv_path)
        images_path = []
        labels = []
        for i, image_path, label in zip(df['class_id'], df["img_path"], df["label"]):
            if is_train and i > 200:
                images_path.append(os.path.join(base_path, image_path))
                labels.append(label)
            else:
                images_path.append(os.path.join(base_path, image_path))
                labels.append(label)
        
        self.classes = sorted(set(labels))
        self.cls2idx = {c:i for i, c in enumerate(classes)}
        self.images_path = images_path
        self.labels = labels
        self.is_train = is_train
        self.tf = tf
    
    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, index):
        image_path = self.images_path[index]
        label = self.labels[labels]

        image = cv2.imread(image_path)
        image = image / 255.0

        if self.tf is not None:
            inputs = ({"image": image})
            result = self.tf(**inputs)

            image = result["image"]
        image = image.transpose(2, 0, 1)
        return image, self.cls2idx[label]

In [6]:
import albumentations as A

tf = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_dataset = CustomDataset(
            "/opt/ml/level3_cv_finalproject-cv-01/model/data/data.csv", is_train=True, tf=tf
        )

In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=8,
            drop_last=True,
        )